In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

In [3]:
file_cols = dict()

In [4]:
test_df = pd.read_csv('test.csv')
file_cols['test'] = test_df.columns
test_df['date'] = pd.to_datetime(test_df['date'])
test_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [5]:
oil_df = pd.read_csv('oil.csv')
file_cols['oil'] = oil_df.columns
oil_df['date'] = pd.to_datetime(oil_df['date'])
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [6]:
train_df = pd.read_csv('train.csv').drop('id', axis=1)
file_cols['train'] = train_df.columns
train_df['date'] = pd.to_datetime(train_df['date']).dt.strftime("%m-%d")
train_df.head()

,date,store_nbr,family,sales,onpromotion
0,01-01,1,AUTOMOTIVE,0.0,0
1,01-01,1,BABY CARE,0.0,0
2,01-01,1,BEAUTY,0.0,0
3,01-01,1,BEVERAGES,0.0,0
4,01-01,1,BOOKS,0.0,0


In [15]:
stores_df = pd.read_csv('stores.csv')
stores_df['nation'] = 'Ecuador'
file_cols['stores'] = stores_df.columns
stores_df.head()

,store_nbr,city,state,type,cluster,nation
0,1,Quito,Pichincha,D,13,Ecuador
1,2,Quito,Pichincha,D,13,Ecuador
2,3,Quito,Pichincha,D,8,Ecuador
3,4,Quito,Pichincha,D,9,Ecuador
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,Ecuador


In [9]:
transaction_df = pd.read_csv('transactions.csv')
file_cols['transaction'] = transaction_df.columns
transaction_df['date'] = pd.to_datetime(transaction_df['date']).dt.strftime('%m-%d')
transaction_df.head()

,date,store_nbr,transactions
0,01-01,25,770
1,01-02,1,2111
2,01-02,2,2358
3,01-02,3,3487
4,01-02,4,1922


In [10]:
holiday_df = pd.read_csv('holidays_events.csv')
file_cols['holiday'] = holiday_df.columns
holiday_df['date'] = pd.to_datetime(holiday_df['date'])
holiday_df.head()
holiday_no_transferred = holiday_df[holiday_df['transferred'] == False]
holiday_no_transferred['date'] = holiday_no_transferred['date'].dt.strftime("%m-%d")
holiday_no_transferred.drop(columns=['description','transferred'], axis=1, inplace=True)
holiday_no_transferred.head()

C:\Users\paulw\AppData\Local\Temp\ipykernel_11496\3660757212.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holiday_no_transferred['date'] = holiday_no_transferred['date'].dt.strftime("%m-%d")
C:\Users\paulw\AppData\Local\Temp\ipykernel_11496\3660757212.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holiday_no_transferred.drop(columns=['description','transferred'], axis=1, inplace=True)


,date,type,locale,locale_name
0,03-02,Holiday,Local,Manta
1,04-01,Holiday,Regional,Cotopaxi
2,04-12,Holiday,Local,Cuenca
3,04-14,Holiday,Local,Libertad
4,04-21,Holiday,Local,Riobamba


In [11]:
file_cols

{'test': Index(['id', 'date', 'store_nbr', 'family', 'onpromotion'], dtype='object'),
 'oil': Index(['date', 'dcoilwtico'], dtype='object'),
 'train': Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion'], dtype='object'),
 'stores': Index(['store_nbr', 'city', 'state', 'type', 'cluster', 'national'], dtype='object'),
 'transaction': Index(['date', 'store_nbr', 'transactions'], dtype='object'),
 'holiday': Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')}

In [12]:
def conbine_holiday_cols(holiday_type):
    
    holiday_type_df = holiday_no_transferred[holiday_no_transferred['type']==holiday_type]
    holiday_type_df = holiday_type_df.groupby(['date', 'locale', 'locale_name'], as_index=False).first()
    
    holiday_local = holiday_type_df[holiday_type_df['locale'] == 'Local']
    holiday_local = holiday_local.drop(columns=['type', 'locale'], axis=1)
    holiday_local[holiday_type+'_local'] = True
    holiday_local.rename(columns={'locale_name':'city'}, inplace=True)
    city_sb_nbr = pd.merge(left=holiday_local, right=stores_df, on='city', how='left')
    local_holiday_nbr = city_sb_nbr[['date','store_nbr',holiday_type+'_local']]
    
    holiday_reg = holiday_type_df[holiday_type_df['locale'] == 'Regional']
    holiday_reg = holiday_reg.drop(columns=['type', 'locale'], axis=1)
    holiday_reg[holiday_type+'_regional'] = True
    holiday_reg.rename(columns={'locale_name':'state'}, inplace=True)
    state_sb_nbr = pd.merge(left=holiday_reg, right=stores_df, on='state', how='left')
    regional_holiday_nbr = state_sb_nbr[['date','store_nbr',holiday_type+'_regional']]

    holiday_national = holiday_type_df[holiday_type_df['locale'] == 'National']
    holiday_national = holiday_national.drop(columns=['type', 'locale'], axis=1)
    holiday_national[holiday_type+'_national'] = True
    holiday_national.rename(columns={'locale_name': 'nation'}, inplace=True)
    nation_sb_nbr = pd.merge(left=holiday_national, right=stores_df, on='nation', how='left')
    national_holiday_nbr = nation_sb_nbr[['date', 'store_nbr', holiday_type+'_national']]

    return (local_holiday_nbr, regional_holiday_nbr, national_holiday_nbr)

In [16]:
(holiday_local, holiday_regional, holiday_national) = conbine_holiday_cols('Holiday')

In [17]:
(addtional_local, addtional_regional, addtional_national) = conbine_holiday_cols('Additional')

In [18]:
(event_local, event_regional, event_national) = conbine_holiday_cols('Event')

In [21]:
addtional_local

,date,store_nbr,Additional_local
0,07-24,24,True
1,07-24,26,True
2,07-24,28,True
3,07-24,29,True
4,07-24,30,True
5,07-24,32,True
6,07-24,34,True
7,07-24,51,True
8,07-25,24,True
9,07-25,26,True


In [20]:
holiday_df[(holiday_df['type'] == 'Additional')&(holiday_df['locale'] == 'National')]

,date,type,locale,locale_name,description,transferred
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False
33,2012-12-22,Additional,National,Ecuador,Navidad-3,False
34,2012-12-23,Additional,National,Ecuador,Navidad-2,False
36,2012-12-24,Additional,National,Ecuador,Navidad-1,False
38,2012-12-26,Additional,National,Ecuador,Navidad+1,False
40,2012-12-31,Additional,National,Ecuador,Primer dia del ano-1,False
53,2013-05-11,Additional,National,Ecuador,Dia de la Madre-1,False
84,2013-12-21,Additional,National,Ecuador,Navidad-4,False
85,2013-12-22,Additional,National,Ecuador,Navidad-3,False
87,2013-12-23,Additional,National,Ecuador,Navidad-2,False


In [46]:
holiday_df['type'].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)